In [1]:
import pandas as pd
from datetime import datetime, date, timedelta

from schedule_musicbox_shows import insert_events_of_mb_shows
from utils import authenticate

import pickle

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [20]:
show_df = pd.read_pickle('../../data/pkl/musicbox/musicbox_showtimes.pkl')
show_df.head()

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
0,DAVID LYNCH: THE ART LIFE,2016,"Jon Nguyen, Rick Barnes",2025-04-18 13:00:00,2025-04-18,13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 14:00:00,2025-04-18,14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 18:45:00,2025-04-18,18:45:00
3,SUNSET BOULEVARD,1950,Billy Wilder,2025-04-18 15:00:00,2025-04-18,15:00:00
4,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 16:15:00,2025-04-18,16:15:00


In [21]:
today = date.today()
n_days_of_shows_to_slate = timedelta(days=2)

# test_show_df = show_df[show_df['Showtime_Date'] >= today].head(10)
# test_show_df

test_show_df = show_df[(show_df['Showtime_Date'] >= today) & (show_df['Showtime_Date'] <= today + n_days_of_shows_to_slate)]
test_show_df

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
77,DAISIES,1966,Vera Chytilová,2025-04-30 19:00:00,2025-04-30,19:00:00
78,MUSIC BOX BINGO,None,None,2025-05-01 19:00:00,2025-05-01,19:00:00
79,CLOWN IN A CORNFIELD,2025,Eli Craig,2025-05-01 19:00:00,2025-05-01,19:00:00


In [22]:
show_df[show_df['Showtime_Date'] <= today + n_days_of_shows_to_slate]

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
0,DAVID LYNCH: THE ART LIFE,2016,"Jon Nguyen, Rick Barnes",2025-04-18 13:00:00,2025-04-18,13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 14:00:00,2025-04-18,14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 18:45:00,2025-04-18,18:45:00
3,SUNSET BOULEVARD,1950,Billy Wilder,2025-04-18 15:00:00,2025-04-18,15:00:00
4,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 16:15:00,2025-04-18,16:15:00
...,...,...,...,...,...,...
75,MUSIC BOX MOVIE TRIVIA,None,None,2025-04-27 16:00:00,2025-04-27,16:00:00
76,DUDE BRO PARTY MASSACRE III,2015,"Tomm Jacobsen, Michael Rousselet, Jon Salmon",2025-04-28 21:00:00,2025-04-28,21:00:00
77,DAISIES,1966,Vera Chytilová,2025-04-30 19:00:00,2025-04-30,19:00:00
78,MUSIC BOX BINGO,None,None,2025-05-01 19:00:00,2025-05-01,19:00:00


In [2]:
creds = authenticate(token_path='../../.secret/token.json',
                     creds_path='../../.secret/G_Cal_Creds.json')
# creds = authenticate()

In [24]:
mb_showtimes_df = None
with open('../../data/pkl/musicbox/musicbox_showtimes.pkl', 'rb') as file:
    mb_showtimes_df = pickle.load(file)

today = date.today()
n_days_of_shows_to_slate = timedelta(days=2)
test_show_df = mb_showtimes_df[mb_showtimes_df['Showtime_Date'] <= today + n_days_of_shows_to_slate]
test_show_df

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
0,DAVID LYNCH: THE ART LIFE,2016,"Jon Nguyen, Rick Barnes",2025-04-18 13:00:00,2025-04-18,13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 14:00:00,2025-04-18,14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 18:45:00,2025-04-18,18:45:00
3,SUNSET BOULEVARD,1950,Billy Wilder,2025-04-18 15:00:00,2025-04-18,15:00:00
4,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 16:15:00,2025-04-18,16:15:00
...,...,...,...,...,...,...
75,MUSIC BOX MOVIE TRIVIA,None,None,2025-04-27 16:00:00,2025-04-27,16:00:00
76,DUDE BRO PARTY MASSACRE III,2015,"Tomm Jacobsen, Michael Rousselet, Jon Salmon",2025-04-28 21:00:00,2025-04-28,21:00:00
77,DAISIES,1966,Vera Chytilová,2025-04-30 19:00:00,2025-04-30,19:00:00
78,MUSIC BOX BINGO,None,None,2025-05-01 19:00:00,2025-05-01,19:00:00


In [ ]:
def check_df_for_search_keys(df: pd.DataFrame, title: str, year: str, director: str) -> bool:
            return not df.loc[(df['Title'] == title) & (df['Showtime'] == year)].empty

In [25]:
try:
        service = build("calendar", "v3", credentials=creds)

        test_cal_id = '806c0e2aac6fb8ce5f7fa446b3f2951b58c055e16a72f3c4e08711b4d813a0bc@group.calendar.google.com'


        scheduled_shows = []
        page_token = None
        while True:
                pages_event_results = service.events().list(calendarId=test_cal_id, pageToken=page_token).execute()

                pages_events = pages_event_results.get('items', [])

                for event in pages_events:
                        show_title = event['summary']
                        show_start = event['start']['dateTime']
                        show_start_dttime = datetime.fromisoformat(show_start).astimezone()
                        show_start_dttime_naive = show_start_dttime.replace(tzinfo=None)


                        show_record = {
                                "Title": show_title,
                                "Showtime": show_start_dttime_naive,
                                }
                        # print(show_record)

                        scheduled_shows.append(show_record)

                page_token = pages_event_results.get('nextPageToken')
                if not page_token:
                        break

except HttpError as error:
        print("An error occurred:", error)

print(scheduled_shows)
scheduled_df = pd.DataFrame(scheduled_shows)
scheduled_df.to_csv('../../data/csv/musicbox/musicbox_scheduled_shows.csv', index=False)
scheduled_df.head()

[{'Title': 'DAVID LYNCH: THE ART LIFE', 'Showtime': datetime.datetime(2025, 4, 18, 13, 0)}, {'Title': 'THE BALLAD OF WALLIS ISLAND', 'Showtime': datetime.datetime(2025, 4, 18, 14, 0)}, {'Title': 'THE BALLAD OF WALLIS ISLAND', 'Showtime': datetime.datetime(2025, 4, 18, 18, 45)}, {'Title': 'SUNSET BOULEVARD', 'Showtime': datetime.datetime(2025, 4, 18, 15, 0)}, {'Title': 'SECRET MALL APARTMENT', 'Showtime': datetime.datetime(2025, 4, 18, 16, 15)}, {'Title': 'SECRET MALL APARTMENT', 'Showtime': datetime.datetime(2025, 4, 18, 21, 0)}, {'Title': 'MULHOLLAND DRIVE', 'Showtime': datetime.datetime(2025, 4, 18, 17, 45)}, {'Title': 'BLUE VELVET', 'Showtime': datetime.datetime(2025, 4, 18, 23, 59)}, {'Title': 'ON THE AIR EPISODES 1 & 7', 'Showtime': datetime.datetime(2025, 4, 19, 11, 30)}, {'Title': 'SECRET MALL APARTMENT', 'Showtime': datetime.datetime(2025, 4, 19, 11, 45)}, {'Title': 'SECRET MALL APARTMENT', 'Showtime': datetime.datetime(2025, 4, 19, 21, 30)}, {'Title': 'THE SHORT FILMS OF DAVID

,Title,Showtime
0,DAVID LYNCH: THE ART LIFE,2025-04-18 13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025-04-18 14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025-04-18 18:45:00
3,SUNSET BOULEVARD,2025-04-18 15:00:00
4,SECRET MALL APARTMENT,2025-04-18 16:15:00


In [27]:
pd.merge(mb_showtimes_df, scheduled_df, how='left', on=['Title', 'Showtime'], indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
178,THE BALTIMORONS,2025,Jay Duplass,2025-05-02 19:00:00,2025-05-02,19:00:00
179,LURKER,None,Alex Russell,2025-05-02 21:45:00,2025-05-02,21:45:00
180,THE CATHEDRAL OF NEW EMOTIONS,2006,Helmut Herbst,2025-05-02 23:59:00,2025-05-02,23:59:00
181,BEST WISHES TO ALL,2023,Yûta Shimotsu,2025-05-02 23:59:00,2025-05-02,23:59:00
182,DON'T LET'S GO TO THE DOGS TONIGHT,2024,Embeth Davidtz,2025-05-03 11:30:00,2025-05-03,11:30:00
...,...,...,...,...,...,...
238,BOUND,1996,"Lana Wachowski, Lilly Wachowski",2025-05-30 19:00:00,2025-05-30,19:00:00
239,THE WATERMELON WOMAN,1996,Cheryl Dunye,2025-05-31 11:30:00,2025-05-31,11:30:00
240,FANTASTIC MR. FOX,2009,Wes Anderson,2025-05-31 11:30:00,2025-05-31,11:30:00
241,FANTASTIC MR. FOX,2009,Wes Anderson,2025-05-31 22:30:00,2025-05-31,22:30:00


In [45]:

def get_first_n_days_of_showtime_df(
        df = pd.DataFrame,
        n_days: int = 5,
        first_show_date: date = None,
        ) -> pd.DataFrame:
    """
    Get the first n days of showtimes from the given DataFrame.
    If first_show_date is not provided, it will be set to the minimum date in the DataFrame.
    """
    
    if not first_show_date:
        first_show_date = mb_showtimes_df['Showtime'].min().date()
    
    n_days_after_show_date = first_show_date + timedelta(days=n_days-1)

    first_n_days_of_shows_df = df[
        (df['Showtime_Date'] >= first_show_date) &
        (df['Showtime_Date'] <= n_days_after_show_date)
        ]
    
    return first_n_days_of_shows_df.reset_index(drop=True)

get_first_n_days_of_showtime_df(
    df=mb_showtimes_df,
    n_days=1,
    )




,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
0,DAVID LYNCH: THE ART LIFE,2016,"Jon Nguyen, Rick Barnes",2025-04-18 13:00:00,2025-04-18,13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 14:00:00,2025-04-18,14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 18:45:00,2025-04-18,18:45:00
3,SUNSET BOULEVARD,1950,Billy Wilder,2025-04-18 15:00:00,2025-04-18,15:00:00
4,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 16:15:00,2025-04-18,16:15:00
5,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 21:00:00,2025-04-18,21:00:00
6,MULHOLLAND DRIVE,2001,David Lynch,2025-04-18 17:45:00,2025-04-18,17:45:00
7,BLUE VELVET,1986,David Lynch,2025-04-18 23:59:00,2025-04-18,23:59:00


In [ ]:
print()

In [ ]:
get_first_n_days_of_showtime_df(
    df=mb_showtimes_df,
    n_days=3,
    )

,Title,Year,Director,Showtime,Showtime_Date,Showtime_Time
0,DAVID LYNCH: THE ART LIFE,2016,"Jon Nguyen, Rick Barnes",2025-04-18 13:00:00,2025-04-18,13:00:00
1,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 14:00:00,2025-04-18,14:00:00
2,THE BALLAD OF WALLIS ISLAND,2025,James Griffiths,2025-04-18 18:45:00,2025-04-18,18:45:00
3,SUNSET BOULEVARD,1950,Billy Wilder,2025-04-18 15:00:00,2025-04-18,15:00:00
4,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 16:15:00,2025-04-18,16:15:00
5,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-18 21:00:00,2025-04-18,21:00:00
6,MULHOLLAND DRIVE,2001,David Lynch,2025-04-18 17:45:00,2025-04-18,17:45:00
7,BLUE VELVET,1986,David Lynch,2025-04-18 23:59:00,2025-04-18,23:59:00
8,ON THE AIR EPISODES 1 & 7,1992,"Mark Frost, David Lynch",2025-04-19 11:30:00,2025-04-19,11:30:00
9,SECRET MALL APARTMENT,2024,Jeremy Workman,2025-04-19 11:45:00,2025-04-19,11:45:00
